#**[HW#2] 로이터 뉴스 카테고리로 분류**
*   RNN /LSTM / GRU를 이용하여 입력된 문장의 의미를 파악하여 46개 중 하나의 카테고리로 분류하는 작업


*   **런타임 유형을 GPU로 변경**



In [ ]:
# 텐서플로 2 버전 선택
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

## Data Setup

*  로이터 뉴스 데이터베이스는 총 11,258개의 뉴스 기사와 46개의 카테고리로 나뉘어진 대용량 텍스트 데이터
*  각 뉴스는 단어 인덱스의 시퀀스로 인코딩되어 있음

In [ ]:
# 로이터 뉴스 데이터셋 불러오기
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.preprocessing import sequence
from keras.utils import np_utils

# seed 값 설정
seed = 0
np.random.seed(seed)
tf.compat.v1.set_random_seed(3)

# 불러온 데이터를 학습셋(80%), 테스트셋(20%)으로 나누기
(X_train, Y_train), (X_test, Y_test) = reuters.load_data(num_words=1000, test_split=0.2)

# 데이터 확인하기
category = np.max(Y_train) + 1
print(category, '카테고리')
print(len(X_train), '학습용 뉴스 기사')
print(len(X_test), '테스트용 뉴스 기사')
print(X_train[0])
print(Y_train[0])

In [ ]:
word_to_index = reuters.get_word_index() #  단어와 그 단어에 부여된 인덱스를 리턴
print(word_to_index)

In [ ]:
index_to_word = {}
for key, value in word_to_index.items():
    index_to_word[value] = key          # 인덱스의 실제 단어를 확인
print(' '.join([index_to_word[X] for X in X_train[0]]))

In [ ]:
print('뉴스 기사의 최대 길이 :{}'.format(max(len(l) for l in X_train)))
print('뉴스 기사의 평균 길이 :{}'.format(sum(map(len, X_train))/len(X_train)))

plt.hist([len(s) for s in X_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
plt.hist([s for s in Y_train], bins=46)
plt.xlabel('categories(Y)')  # 뉴스 카테고리별 데이터 수(46개의 카테고리)
plt.ylabel('number of samples')
plt.show()

## **1. LSTM**

In [ ]:
# 데이터 전처리

x_train = sequence.pad_sequences(X_train, maxlen=100)  # maxlen 설정하기(손실이 적어지게)
x_test = sequence.pad_sequences(X_test, maxlen=100)
y_train = np_utils.to_categorical(Y_train)
y_test = np_utils.to_categorical(Y_test)

In [ ]:
# 모델설정(LSTM 1개)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(11258, 300, input_length=2000),
    tf.keras.layers.LSTM(units=300),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(46, activation='softmax')
])

model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
# 모델설정(LSTM 2개)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(11258, 300, input_length=2000),
    tf.keras.layers.LSTM(units=50,return_sequences=True),
    tf.keras.layers.LSTM(units=50),
   # tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(46, activation='softmax')
])

model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
# 모델의 실행
history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], 'g-', label='accuracy')
plt.plot(history.history['val_accuracy'], 'k--', label='val_accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.show()

In [ ]:
# 테스트 정확도 출력
print("\n Test Accuracy: %.4f" % (model.evaluate(x_test, y_test)[1]))

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print(10*(accuracy-loss+1))
print(accuracy)
print(loss)

## **2. GRU**

In [ ]:
# GRU
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(11258, 300, input_length=200),
    #tf.keras.layers.GRU(units=50,return_sequences=True),
    tf.keras.layers.GRU(units=200),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(46, activation='softmax')
])

model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
# 모델의 실행
history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], 'g-', label='accuracy')
plt.plot(history.history['val_accuracy'], 'k--', label='val_accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.show()

In [ ]:
# 테스트 정확도 출력
print("\n Test Accuracy: %.4f" % (model.evaluate(x_test, y_test)[1]))
loss, acc = model.evaluate(x_test, y_test)
print(10*(acc-loss+1))
print(loss)